In [25]:
import _mypath
from bripipetools.util import string_ops as strings
import os, sys, re

In [5]:
!yes | pip install GEOparse

    100% |████████████████████████████████| 9.0MB 75kB/s 
    100% |████████████████████████████████| 3.7MB 184kB/s 
    100% |████████████████████████████████| 481kB 1.5MB/s 
    100% |████████████████████████████████| 204kB 3.1MB/s 
^C


In [6]:
import GEOparse

In [11]:
# create temp directory for storing data
tmp_dir = './tmp'
if not os.path.isdir(tmp_dir):
    os.mkdir('./tmp')

In [12]:
# download GSE SOFT file
acc = 'GSE71698'
gse = GEOparse.get_GEO(geo=acc, destdir=tmp_dir)

Parsing ./tmp/GSE71698.soft.gz:
 - DATABASE : GeoMiame
 - SERIES : GSE71698
 - PLATFORM : GPL11002
 - PLATFORM : GPL13112
 - PLATFORM : GPL17021
 - SAMPLE : GSM1843335
 - SAMPLE : GSM1843336
 - SAMPLE : GSM1843337
 - SAMPLE : GSM1843338
 - SAMPLE : GSM1843339
 - SAMPLE : GSM1843340
 - SAMPLE : GSM1843341
 - SAMPLE : GSM1843342
 - SAMPLE : GSM1843343
 - SAMPLE : GSM1843344
 - SAMPLE : GSM1843345
 - SAMPLE : GSM1843346
 - SAMPLE : GSM1843347
 - SAMPLE : GSM1843348
 - SAMPLE : GSM1843349
 - SAMPLE : GSM1843350
 - SAMPLE : GSM1843351
 - SAMPLE : GSM1843352
 - SAMPLE : GSM1843353
 - SAMPLE : GSM1843354
 - SAMPLE : GSM1843355
 - SAMPLE : GSM1843356
 - SAMPLE : GSM1843357
 - SAMPLE : GSM1843358
 - SAMPLE : GSM1843359
 - SAMPLE : GSM1843360
 - SAMPLE : GSM1843361
 - SAMPLE : GSM1843362
 - SAMPLE : GSM1843363
 - SAMPLE : GSM1843364
 - SAMPLE : GSM1843365
 - SAMPLE : GSM1843366
 - SAMPLE : GSM1843367
 - SAMPLE : GSM1843368
 - SAMPLE : GSM1843369
 - SAMPLE : GSM1843370
 - SAMPLE : GSM1904864
 - S

In [82]:
import shutil
import urllib2
from contextlib import closing

def get_supplementary_files(gse):
    supp_file_dict = {}

    # loop through all GSM samples
    for gsm_name, gsm in gse.gsms.iteritems():
        
        # parse metadata items to find supplementary files
        for key, value in gsm.metadata.iteritems(): 
            if re.search('supplementary_file', key):
                # if the supplementary file has a file extension, add to dictionary;
                # otherwise, ignore (requires alternative collection)
                supp_type = strings.matchdefault('\..*', os.path.basename(value[0]))
                if len(supp_type):
                    supp_file_dict.setdefault(supp_type, []).append(value[0])
    
    return supp_file_dict

def select_supplementary_filetype(keys):
    print "\nFound the following supplemenmtary filetypes:"
    for i, k in enumerate(keys):
        print("%3d : %s" % (i, k))

    k_i = raw_input("\nType the number of the filetype you wish to collect or hit enter to finish: ")
    # TODO: add the option (while loop) to add additional filetypes
    if len(k_i):
        selected_key = keys[int(k_i)]

    return selected_key

        
def collect_supplementary_files(gse):    
    supp_file_dict = get_supplementary_files(gse)
    selected_type = select_supplementary_filetype(supp_file_dict.keys())
    
    supplementary_files = supp_file_dict[selected_type]
    print("\nCollecting %d supplementary files with extension '%s'..." 
          % (len(supplementary_files), selected_type))
    
    for supp_f in supplementary_files:
        gsm_name = strings.matchdefault('GSM[0-9]+', supp_f)
        print('- Downloading file for %s...' % gsm_name),

        target_f = os.path.join(tmp_dir, os.path.basename(supp_f))
        with closing(urllib2.urlopen(supp_f)) as r:
            with open(target_f, 'wb') as f:
                shutil.copyfileobj(r, f)
        print('done.')
        break
    
collect_supplementary_files(gse)

 
Found the following supplemenmtary filetypes:
  0 : .gc_norm_rpkm.txt.gz
  1 : .62M37AAXX_7_20110728a.sorted.cov.all.bw
  2 : .sorted.0mer.cov.all.bw
  3 : .sorted.cov.all.bw
  4 : .rpkm.txt.gz

Type the number of the filetype you wish to collect or hit enter to finish: 0

- Downloading file for GSM1843... done.


In [92]:
f = 'ftp://ftp.ncbi.nlm.nih.gov/pub/geo/DATA/supplementary/samples/GSM1843nnn/GSM1843364/GSM1843364_11952.gc_norm_rpkm.txt.gz'
strings.matchdefault('GSM[0-9]+(?=[^GSM]*$)', f)

'GSM1843364'